# Trees in Paris

In [ ]:
# Extract compressed file
import tarfile

with tarfile.open("trees.tar.gz", "r:gz") as tar:
    tar.extractall()

In [ ]:
from json import load
import pandas as pd

from ipywidgets import Text, VBox
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale

from py2vega.functions.color import rgb

from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr

with open("./trees.json") as fobj:
    data = load(fobj)

df = pd.DataFrame(data["data"]).drop("index", axis=1)
df = df[sorted(df.columns)]

# Get list of possible tree species and kinds for the ordinal color scales
species = list(
    set(
        [
            tree["Species"]
            for tree in data["data"]
            if tree["Species"] is not None
        ]
    )
)
kinds = list(
    set([tree["Kind"] for tree in data["data"] if tree["Kind"] is not None])
)

# Text box for highlighting tree names
highlight_box = Text(description="Highlight: ", value="Platane")
name_background_color = Expr(
    "'lightgreen' if '{}' in cell.value else default_value".format(
        highlight_box.value
    )
)


def on_textbox_changed(*args, **kwargs):
    name_background_color.value = (
        "'lightgreen' if '{}' in cell.value else default_value".format(
            highlight_box.value
        )
    )


highlight_box.observe(on_textbox_changed, "value")


def circumference_color(cell):
    color_value = 255 if cell.value > 300 else (cell.value / 300) * 255
    return rgb(0, color_value, color_value)


renderers = {
    "French_name": TextRenderer(background_color=name_background_color),
    "Height_m": BarRenderer(
        bar_value=LinearScale(min=0, max=30),
        bar_color=ColorScale(min=0, max=30, scheme="viridis"),
        horizontal_alignment="center",
    ),
    "Species": TextRenderer(
        text_color="black", background_color=OrdinalColorScale(domain=species)
    ),
    "Kind": TextRenderer(
        text_color="black", background_color=OrdinalColorScale(domain=kinds)
    ),
    "Type": TextRenderer(missing="Unspecified"),
    "Circumference_cm": BarRenderer(
        bar_value=LinearScale(min=0, max=300),
        bar_color=Expr(circumference_color),
        horizontal_alignment="center",
    ),
}

datagrid = DataGrid(
    df, base_row_size=32, base_column_size=90, renderers=renderers
)
VBox((highlight_box, datagrid))

In [ ]:
def circumference_color(cell):
    color_value = 255 if cell.value > 300 else (cell.value / 300) * 255
    return rgb(color_value, 0, 0)


renderers["Circumference_cm"].bar_color.value = circumference_color

In [ ]:
datagrid.transform(
    [
        {"type": "sort", "columnIndex": 5},
        {
            "type": "filter",
            "operator": "=",
            "columnIndex": 4,
            "value": "Platane",
        },
    ]
)